In [1]:
import pandas as pd

if not 'precision' in locals():
  precision = 'allele'

if not 'test_path' in locals():
  test_path = f'/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/{precision}/paired/test.tsv'

if not 'train_path' in locals():
  train_path = f'/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/{precision}/paired/train.tsv'

if not 'validation_path' in locals():
  validation_path = f'/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/{precision}/paired/validation.tsv'

if not 'output_path' in locals():
  output_path = f'/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/{precision}/paired/test_reclassified_paired_specific.tsv'

if not 'paired_data_path' in locals():
  paired_data_path =f'/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/{precision}/paired/'

if not 'alpha_cdr3_name' in locals():
  alpha_cdr3_name = 'TRA_CDR3'

if not 'beta_cdr3_name' in locals():
  beta_cdr3_name = 'TRB_CDR3'

if not 'epitope_name' in locals():
  epitope_name = 'Epitope'

if not 'task_name' in locals():
  task_name = 'task'

print(precision)

allele


In [2]:
print(train_path)
df_train = pd.read_csv(train_path, sep="\t", index_col=False)
df_train_old = df_train.copy()

df_test = pd.read_csv(test_path, sep="\t", index_col=False)
df_test_old = df_test.copy()

df_validate = pd.read_csv(validation_path, sep="\t", index_col=False)
df_validate_old = df_validate.copy()

df_seen = pd.concat([df_train, df_validate]) # epitopes and cdr3 are seen if in validate or train.
df_seen_old = df_seen.copy()
df_test

/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/allele/paired/train.tsv


,TCR_name,TRAV,TRAJ,TRA_CDR3,TRBV,TRBJ,TRB_CDR3,TRAC,TRBC,Epitope,MHC,Binding,task
0,1,TRAV29/DV5*01,TRAJ37*01,CAASAEGNTGKLIF,TRBV2*01,TRBJ2-7*01,CASSDPGARVWEQYF,NaN,NaN,TTDPSFLGRY,HLA-A*01:01,1,TPP1
1,2,TRAV3,NaN,CAVRDDTGGFKTIF,TRBV19,NaN,CASSTTEAYEQYF,NaN,NaN,IMDQVPFSV,HLA-A*02:01,1,TPP1
2,3,TRAV13-1*01,TRAJ8*01,CAASKGTGFQKLVF,TRBV4-2*01,TRBJ2-7*01,CASSHPSEGSRKKYEQYF,NaN,NaN,YLQPRTFLL,HLA-A*02:01,1,TPP1
3,4,TRAV13-1*01,TRAJ42*01,CAARTMEGSQGNLIF,TRBV9*01,TRBJ2-7*01,CASSVAPGEYEQYF,NaN,NaN,KLGGALQAK,HLA-A*03:01,1,TPP1
4,5,NaN,NaN,CAGHTGNQFYF,NaN,NaN,CASSLRDGSEAFF,NaN,NaN,IVTDFSVIK,HLA-A*11:01,1,TPP1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15604,15605,TRAV38-2/DV8*01,TRAJ48*01,CADFGNEKLTF,TRBV20-1*01,TRBJ2-5*01,CSAREWAWEETQYF,NaN,NaN,KLWASPLHV,HLA-A*02:01,0,TPP2
15605,15606,TRAV3*01,TRAJ26*01,CAVRAYGQNFVF,TRBV4-3*01,TRBJ2-3*01,CASSQTPPGQGTDTQYF,NaN,NaN,LTDEMIAQY,HLA-A*01:01,0,TPP2
15606,15607,TRAV12-3,TRAJ45,CAKPTGGGADGLTF,TRBV3-1,TRBJ2-2,CASSQDMRDANTGELFF,NaN,NaN,FLIYLDVSV,HLA-A*02:01,0,TPP2
15607,15608,TRAV12-2,NaN,CAADSWGKLQF,TRBV24-1,NaN,CATSDSTGSYGYTF,NaN,NaN,KLGGALQAK,HLA-A*03:01,0,TPP2


In [3]:
alpha_cdr3_in_train = set(df_seen[alpha_cdr3_name])
beta_cdr3_in_train = set(df_seen[beta_cdr3_name])
epitopes_in_train = set(df_seen[epitope_name])

In [4]:
# Function to verify each row
def calculate_task(row):
    epitope_exists = row[epitope_name] in epitopes_in_train
    alpha_cdr3_exists = row[alpha_cdr3_name] in alpha_cdr3_in_train
    beta_cdr3_exists = row[beta_cdr3_name] in beta_cdr3_in_train

    cdr3_exists = alpha_cdr3_exists or beta_cdr3_exists

    if epitope_exists and cdr3_exists:
        return 'TPP1'
    elif epitope_exists and not cdr3_exists:
        return 'TPP2'
    elif not epitope_exists and not cdr3_exists:
        return 'TPP3'
    elif not epitope_exists and cdr3_exists:
        return 'TPP4'
    raise Exception("Something seems wrong")  # This handles unexpected cases
    
# Apply the verification function
print(df_test)
df_test[task_name] = df_test.apply(lambda x: calculate_task(x), axis=1)

       TCR_name             TRAV       TRAJ         TRA_CDR3         TRBV  \
0             1    TRAV29/DV5*01  TRAJ37*01   CAASAEGNTGKLIF     TRBV2*01   
1             2            TRAV3        NaN   CAVRDDTGGFKTIF       TRBV19   
2             3      TRAV13-1*01   TRAJ8*01   CAASKGTGFQKLVF   TRBV4-2*01   
3             4      TRAV13-1*01  TRAJ42*01  CAARTMEGSQGNLIF     TRBV9*01   
4             5              NaN        NaN      CAGHTGNQFYF          NaN   
...         ...              ...        ...              ...          ...   
15604     15605  TRAV38-2/DV8*01  TRAJ48*01      CADFGNEKLTF  TRBV20-1*01   
15605     15606         TRAV3*01  TRAJ26*01     CAVRAYGQNFVF   TRBV4-3*01   
15606     15607         TRAV12-3     TRAJ45   CAKPTGGGADGLTF      TRBV3-1   
15607     15608         TRAV12-2        NaN      CAADSWGKLQF     TRBV24-1   
15608     15609        TRAV17*01  TRAJ44*01     CASTGTASKLTF     TRBV9*01   

             TRBJ            TRB_CDR3  TRAC  TRBC     Epitope          MHC 

In [5]:
number_of_TPP1 = (df_test['task'] == 'TPP1').sum()
number_of_TPP1_old = (df_test_old['task'] == 'TPP1').sum()
number_of_TPP2 = (df_test['task'] == 'TPP2').sum()
number_of_TPP2_old = (df_test_old['task'] == 'TPP2').sum()
number_of_TPP3 = (df_test['task'] == 'TPP3').sum()
number_of_TPP3_old = (df_test_old['task'] == 'TPP3').sum()
number_of_TPP4 = (df_test['task'] == 'TPP4').sum()
number_of_TPP4_old = (df_test_old['task'] == 'TPP4').sum()
test_ratio = len(df_test)/(len(df_test) + len(df_seen))

print(f"train+validate data has {len(df_seen)} entries")
print(f"test data has {len(df_test)} entries")
print(f"test data has {number_of_TPP1} TPP1 tasks (old value: {number_of_TPP1_old}) (seen tcr & seen epitopes).")
print(f"test data has {number_of_TPP2} TPP2 tasks (old value: {number_of_TPP2_old}) (unseen tcr & seen epitopes).")
print(f"test data has {number_of_TPP3} TPP3 tasks (old value: {number_of_TPP3_old}) (unseen tcr & unseen epitope).")
print(f"test data has {number_of_TPP4} TPP4 tasks (old value: {number_of_TPP4_old}) (seen tcr & unseen epitope).")
print(f"the train/test ratio is {(1-test_ratio)}/{test_ratio}")
df_test

train+validate data has 88231 entries
test data has 15609 entries
test data has 10949 TPP1 tasks (old value: 7434) (seen tcr & seen epitopes).
test data has 4440 TPP2 tasks (old value: 7955) (unseen tcr & seen epitopes).
test data has 100 TPP3 tasks (old value: 145) (unseen tcr & unseen epitope).
test data has 120 TPP4 tasks (old value: 75) (seen tcr & unseen epitope).
the train/test ratio is 0.8496822033898305/0.1503177966101695


,TCR_name,TRAV,TRAJ,TRA_CDR3,TRBV,TRBJ,TRB_CDR3,TRAC,TRBC,Epitope,MHC,Binding,task
0,1,TRAV29/DV5*01,TRAJ37*01,CAASAEGNTGKLIF,TRBV2*01,TRBJ2-7*01,CASSDPGARVWEQYF,NaN,NaN,TTDPSFLGRY,HLA-A*01:01,1,TPP1
1,2,TRAV3,NaN,CAVRDDTGGFKTIF,TRBV19,NaN,CASSTTEAYEQYF,NaN,NaN,IMDQVPFSV,HLA-A*02:01,1,TPP1
2,3,TRAV13-1*01,TRAJ8*01,CAASKGTGFQKLVF,TRBV4-2*01,TRBJ2-7*01,CASSHPSEGSRKKYEQYF,NaN,NaN,YLQPRTFLL,HLA-A*02:01,1,TPP1
3,4,TRAV13-1*01,TRAJ42*01,CAARTMEGSQGNLIF,TRBV9*01,TRBJ2-7*01,CASSVAPGEYEQYF,NaN,NaN,KLGGALQAK,HLA-A*03:01,1,TPP1
4,5,NaN,NaN,CAGHTGNQFYF,NaN,NaN,CASSLRDGSEAFF,NaN,NaN,IVTDFSVIK,HLA-A*11:01,1,TPP1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15604,15605,TRAV38-2/DV8*01,TRAJ48*01,CADFGNEKLTF,TRBV20-1*01,TRBJ2-5*01,CSAREWAWEETQYF,NaN,NaN,KLWASPLHV,HLA-A*02:01,0,TPP2
15605,15606,TRAV3*01,TRAJ26*01,CAVRAYGQNFVF,TRBV4-3*01,TRBJ2-3*01,CASSQTPPGQGTDTQYF,NaN,NaN,LTDEMIAQY,HLA-A*01:01,0,TPP2
15606,15607,TRAV12-3,TRAJ45,CAKPTGGGADGLTF,TRBV3-1,TRBJ2-2,CASSQDMRDANTGELFF,NaN,NaN,FLIYLDVSV,HLA-A*02:01,0,TPP2
15607,15608,TRAV12-2,NaN,CAADSWGKLQF,TRBV24-1,NaN,CATSDSTGSYGYTF,NaN,NaN,KLGGALQAK,HLA-A*03:01,0,TPP2


In [6]:
print(output_path)
df_test.to_csv(output_path, sep="\t", index=False)

/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/allele/paired/test_reclassified_paired_specific.tsv


In [7]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv()

# upload paired data
path_to_data = paired_data_path
dataset_name = f'paired_{precision}'
main_project_name = os.getenv("MAIN_PROJECT_NAME")

%run /teamspace/studios/this_studio/BA_ZHAW/data_scripts/upload_datasets.ipynb


uploading dataset to BA_Project_ZHAW


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gabricyr (ba-zhaw). Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (/teamspace/studios/this_studio/BA_ZHAW/data/splitted_datasets/allele/paired)... Done. 0.1s
